# Check if everything is in place

## Weaviate Python Client v4
> This notebook was created with Weaviate `1.26` and the Weaviate Client `4.7`

Run the below command to check if you run the latest version of the Weaviate Python Client v4.

In [ ]:
!pip show weaviate-client

## Get keys and urls

> You can update your env variables in the `.env` file at the root of the project.

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

WEAVIATE_URL = os.getenv("WEAVIATE_URL")
WEAVIATE_KEY = os.getenv("WEAVIATE_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

print(WEAVIATE_URL)
print(WEAVIATE_KEY)
print(OPENAI_API_KEY)

## Connect to Weaviate

In [ ]:
import weaviate
from weaviate.classes.init import Auth
# from weaviate.classes.init import AdditionalConfig, Timeout

client = weaviate.connect_to_weaviate_cloud(
    cluster_url=WEAVIATE_URL,
    auth_credentials=Auth.api_key(WEAVIATE_KEY),

    # additional_config=AdditionalConfig(
    #     timeout=Timeout(init=2, query=45, insert=120),  # Values in seconds
    # )
)

client.is_ready()

## Display the available modules

> You should be able to see 'generative-openai' and 'text2vec-openai', plus many other modules.

In [ ]:
client.get_meta()

### Close the client
When you are done with the client, you should close it to release the resources.

In [ ]:
client.close()